In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [4]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    param1=["best","random"]
    score=0
    best_param=param1[0]
    for i in param1:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        if ACC>score:
            score=ACC
            best_param=i
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param1,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
    return best_param,results

In [5]:
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    score=0
    best_param=param2[0]
    #param1=["best","random"]
    for i in param2:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        if ACC>score:
            score=ACC
            best_param=i
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param2,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
  #  return grid_search.best_params_["splitter"],results
    return best_param,results

In [6]:
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,50))  ## 设置max_features范围
  #  param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    #param1=["best","random"]
    for j in param3:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=j)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param3,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    return results

In [7]:
file_path = 'E:\datatest\PseDPC\d4\d41'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
411.csv
**************************************************
2022-02-24 16:33:26.923964
2022-02-24 16:33:29.597810
2022-02-24 16:33:42.620967
##################################################
412.csv
**************************************************
2022-02-24 16:33:47.625578
2022-02-24 16:33:50.174757
2022-02-24 16:34:02.799009
##################################################
413.csv
**************************************************
2022-02-24 16:34:06.898026
2022-02-24 16:34:09.995726
2022-02-24 16:34:14.378029
##################################################
414.csv
**************************************************
2022-02-24 16:34:15.904916
2022-02-24 16:34:18.292528
2022-02-24 16:34:30.034113
##################################################
415.csv
**************************************************
2022-02-24 16:34:34.344580
2022-02-24 16:34:36.819958
2022-02-24 16:34:41.003764
#############################################

In [8]:
file_path = 'E:\datatest\PseDPC\d4\d42'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
421.csv
**************************************************
2022-02-24 16:35:52.881456
2022-02-24 16:35:55.292007
2022-02-24 16:36:07.001677
##################################################
422.csv
**************************************************
2022-02-24 16:36:12.087072
2022-02-24 16:36:14.568432
2022-02-24 16:36:26.466599
##################################################
423.csv
**************************************************
2022-02-24 16:36:30.622481
2022-02-24 16:36:33.190609
2022-02-24 16:36:45.781921
##################################################
424.csv
**************************************************
2022-02-24 16:36:50.937129
2022-02-24 16:36:53.483316
2022-02-24 16:37:05.898101
##################################################
425.csv
**************************************************
2022-02-24 16:37:10.900716
2022-02-24 16:37:13.326226
2022-02-24 16:37:25.563486
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d4\d43'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
431.csv
**************************************************
2022-02-24 16:38:48.474657
2022-02-24 16:38:50.769517
2022-02-24 16:39:02.257782
##################################################
432.csv
**************************************************
2022-02-24 16:39:07.069906
2022-02-24 16:39:09.415630
2022-02-24 16:39:20.829093
##################################################
433.csv
**************************************************
2022-02-24 16:39:25.776855
2022-02-24 16:39:28.208350
2022-02-24 16:39:40.058645
##################################################
434.csv
**************************************************
2022-02-24 16:39:45.091180
2022-02-24 16:39:47.640360
2022-02-24 16:39:59.880611
##################################################
435.csv
**************************************************
2022-02-24 16:40:04.874251
2022-02-24 16:40:07.506209
2022-02-24 16:40:20.706890
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d4\d44'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
441.csv
**************************************************
2022-02-24 16:41:33.226864
2022-02-24 16:41:35.927639
2022-02-24 16:41:40.520351
##################################################
442.csv
**************************************************
2022-02-24 16:41:42.985754
2022-02-24 16:41:45.510001
2022-02-24 16:41:49.656905
##################################################
443.csv
**************************************************
2022-02-24 16:41:51.738336
2022-02-24 16:41:54.433126
2022-02-24 16:42:08.127487
##################################################
444.csv
**************************************************
2022-02-24 16:42:12.346200
2022-02-24 16:42:14.998105
2022-02-24 16:42:28.738343
##################################################
445.csv
**************************************************
2022-02-24 16:42:33.745945
2022-02-24 16:42:36.452704
2022-02-24 16:42:50.244802
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d4\d45'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
451.csv
**************************************************
2022-02-24 16:44:23.616986
2022-02-24 16:44:26.226005
2022-02-24 16:44:39.611195
##################################################
452.csv
**************************************************
2022-02-24 16:44:45.114469
2022-02-24 16:44:47.804273
2022-02-24 16:44:52.056896
##################################################
453.csv
**************************************************
2022-02-24 16:44:53.795244
2022-02-24 16:44:56.373346
2022-02-24 16:45:00.470385
##################################################
454.csv
**************************************************
2022-02-24 16:45:02.250623
2022-02-24 16:45:05.009241
2022-02-24 16:45:19.029729
##################################################
455.csv
**************************************************
2022-02-24 16:45:23.269386
2022-02-24 16:45:26.030000
2022-02-24 16:45:39.910862
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d4\d46'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
461.csv
**************************************************
2022-02-24 16:47:09.188999
2022-02-24 16:47:11.904732
2022-02-24 16:47:25.088459
##################################################
462.csv
**************************************************
2022-02-24 16:47:29.956435
2022-02-24 16:47:32.741983
2022-02-24 16:47:46.663736
##################################################
463.csv
**************************************************
2022-02-24 16:47:50.859510
2022-02-24 16:47:53.540337
2022-02-24 16:48:06.667216
##################################################
464.csv
**************************************************
2022-02-24 16:48:12.232327
2022-02-24 16:48:15.045800
2022-02-24 16:48:29.065290
##################################################
465.csv
**************************************************
2022-02-24 16:48:34.728139
2022-02-24 16:48:37.542609
2022-02-24 16:48:51.558111
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d4\d47'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
471.csv
**************************************************
2022-02-24 16:50:25.491792
2022-02-24 16:50:27.965175
2022-02-24 16:50:39.831426
##################################################
472.csv
**************************************************
2022-02-24 16:50:44.021216
2022-02-24 16:50:46.940406
2022-02-24 16:51:01.484494
##################################################
473.csv
**************************************************
2022-02-24 16:51:05.734124
2022-02-24 16:51:08.594471
2022-02-24 16:51:22.678789
##################################################
474.csv
**************************************************
2022-02-24 16:51:26.974296
2022-02-24 16:51:29.798739
2022-02-24 16:51:43.722486
##################################################
475.csv
**************************************************
2022-02-24 16:51:47.963140
2022-02-24 16:51:50.772624
2022-02-24 16:52:04.748232
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d4\d48'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
481.csv
**************************************************
2022-02-24 16:53:32.989143
2022-02-24 16:53:35.662989
2022-02-24 16:53:48.869655
##################################################
482.csv
**************************************************
2022-02-24 16:53:54.422798
2022-02-24 16:53:57.011871
2022-02-24 16:54:09.596202
##################################################
483.csv
**************************************************
2022-02-24 16:54:14.602806
2022-02-24 16:54:17.217810
2022-02-24 16:54:29.858989
##################################################
484.csv
**************************************************
2022-02-24 16:54:34.108619
2022-02-24 16:54:36.699687
2022-02-24 16:54:49.428630
##################################################
485.csv
**************************************************
2022-02-24 16:54:53.635375
2022-02-24 16:54:56.516669
2022-02-24 16:55:10.726647
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d4\d49'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
491.csv
**************************************************
2022-02-24 16:56:47.276329
2022-02-24 16:56:50.084816
2022-02-24 16:57:03.486989
##################################################
492.csv
**************************************************
2022-02-24 16:57:07.497259
2022-02-24 16:57:10.335665
2022-02-24 16:57:24.462868
##################################################
493.csv
**************************************************
2022-02-24 16:57:30.074853
2022-02-24 16:57:32.783605
2022-02-24 16:57:46.313407
##################################################
494.csv
**************************************************
2022-02-24 16:57:50.617890
2022-02-24 16:57:53.435352
2022-02-24 16:58:07.619403
##################################################
495.csv
**************************************************
2022-02-24 16:58:13.214433
2022-02-24 16:58:16.067799
2022-02-24 16:58:30.104245
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d4\d410'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4101.csv
**************************************************
2022-02-24 16:59:59.717484
2022-02-24 17:00:02.720450
2022-02-24 17:00:17.832020
##################################################
4102.csv
**************************************************
2022-02-24 17:00:22.045745
2022-02-24 17:00:24.607891
2022-02-24 17:00:37.525330
##################################################
4103.csv
**************************************************
2022-02-24 17:00:41.717116
2022-02-24 17:00:44.445814
2022-02-24 17:00:58.117237
##################################################
4104.csv
**************************************************
2022-02-24 17:01:02.300046
2022-02-24 17:01:05.023759
2022-02-24 17:01:09.425980
##################################################
4105.csv
**************************************************
2022-02-24 17:01:11.400697
2022-02-24 17:01:14.198213
2022-02-24 17:01:27.939448
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d4\d411'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4111.csv
**************************************************
2022-02-24 17:02:55.018468
2022-02-24 17:02:57.572635
2022-02-24 17:03:10.519995
##################################################
4112.csv
**************************************************
2022-02-24 17:03:15.960438
2022-02-24 17:03:19.184812
2022-02-24 17:03:34.841923
##################################################
4113.csv
**************************************************
2022-02-24 17:03:39.419673
2022-02-24 17:03:42.250100
2022-02-24 17:03:55.965405
##################################################
4114.csv
**************************************************
2022-02-24 17:04:01.400862
2022-02-24 17:04:04.108617
2022-02-24 17:04:17.554643
##################################################
4115.csv
**************************************************
2022-02-24 17:04:22.977135
2022-02-24 17:04:25.829503
2022-02-24 17:04:40.104312
########################################

In [18]:
file_path = 'E:\datatest\PseDPC\d4\d412'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4121.csv
**************************************************
2022-02-24 17:06:07.281070
2022-02-24 17:06:09.974863
2022-02-24 17:06:23.221422
##################################################
4122.csv
**************************************************
2022-02-24 17:06:28.408544
2022-02-24 17:06:31.519222
2022-02-24 17:06:46.043362
##################################################
4123.csv
**************************************************
2022-02-24 17:06:50.314934
2022-02-24 17:06:53.333857
2022-02-24 17:07:08.274882
##################################################
4124.csv
**************************************************
2022-02-24 17:07:12.540470
2022-02-24 17:07:15.689045
2022-02-24 17:07:31.295291
##################################################
4125.csv
**************************************************
2022-02-24 17:07:35.573844
2022-02-24 17:07:38.575812
2022-02-24 17:07:53.497899
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d4\d413'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4131.csv
**************************************************
2022-02-24 17:09:34.260298
2022-02-24 17:09:37.919509
2022-02-24 17:09:52.822636
##################################################
4132.csv
**************************************************
2022-02-24 17:09:57.352515
2022-02-24 17:10:00.735465
2022-02-24 17:10:18.628643
##################################################
4133.csv
**************************************************
2022-02-24 17:10:24.772206
2022-02-24 17:10:28.084345
2022-02-24 17:10:44.169309
##################################################
4134.csv
**************************************************
2022-02-24 17:10:48.554576
2022-02-24 17:10:51.835799
2022-02-24 17:10:56.506302
##################################################
4135.csv
**************************************************
2022-02-24 17:10:58.908873
2022-02-24 17:11:02.020548
2022-02-24 17:11:17.643749
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d4\d414'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4141.csv
**************************************************
2022-02-24 17:12:59.233944
2022-02-24 17:13:02.010516
2022-02-24 17:13:15.721832
##################################################
4142.csv
**************************************************
2022-02-24 17:13:21.637006
2022-02-24 17:13:24.911246
2022-02-24 17:13:40.920413
##################################################
4143.csv
**************************************************
2022-02-24 17:13:47.076941
2022-02-24 17:13:50.337218
2022-02-24 17:14:06.555825
##################################################
4144.csv
**************************************************
2022-02-24 17:14:11.576393
2022-02-24 17:14:15.682924
2022-02-24 17:14:22.173560
##################################################
4145.csv
**************************************************
2022-02-24 17:14:23.877998
2022-02-24 17:14:28.364994
2022-02-24 17:14:45.127146
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d4\d415'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4151.csv
**************************************************
2022-02-24 17:16:13.761006
2022-02-24 17:16:16.982388
2022-02-24 17:16:32.706319
##################################################
4152.csv
**************************************************
2022-02-24 17:16:38.074955
2022-02-24 17:16:41.100859
2022-02-24 17:16:55.446478
##################################################
4153.csv
**************************************************
2022-02-24 17:17:01.235988
2022-02-24 17:17:04.260894
2022-02-24 17:17:19.226853
##################################################
4154.csv
**************************************************
2022-02-24 17:17:24.877735
2022-02-24 17:17:27.920593
2022-02-24 17:17:43.152839
##################################################
4155.csv
**************************************************
2022-02-24 17:17:49.287427
2022-02-24 17:17:52.309342
2022-02-24 17:18:07.050900
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d4\d416'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4161.csv
**************************************************
2022-02-24 17:19:37.694384
2022-02-24 17:19:40.639505
2022-02-24 17:19:55.136719
##################################################
4162.csv
**************************************************
2022-02-24 17:20:00.955150
2022-02-24 17:20:03.932185
2022-02-24 17:20:18.375542
##################################################
4163.csv
**************************************************
2022-02-24 17:20:24.201953
2022-02-24 17:20:27.132114
2022-02-24 17:20:41.548543
##################################################
4164.csv
**************************************************
2022-02-24 17:20:47.564448
2022-02-24 17:20:50.502586
2022-02-24 17:21:04.991820
##################################################
4165.csv
**************************************************
2022-02-24 17:21:10.854135
2022-02-24 17:21:13.789282
2022-02-24 17:21:28.179782
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d4\d417'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4171.csv
**************************************************
2022-02-24 17:23:14.567684
2022-02-24 17:23:18.306681
2022-02-24 17:23:23.711221
##################################################
4172.csv
**************************************************
2022-02-24 17:23:26.137729
2022-02-24 17:23:29.332182
2022-02-24 17:23:46.955067
##################################################
4173.csv
**************************************************
2022-02-24 17:23:52.852289
2022-02-24 17:23:57.166745
2022-02-24 17:24:14.266994
##################################################
4174.csv
**************************************************
2022-02-24 17:24:18.925530
2022-02-24 17:24:22.234676
2022-02-24 17:24:39.543368
##################################################
4175.csv
**************************************************
2022-02-24 17:24:44.845183
2022-02-24 17:24:48.498408
2022-02-24 17:25:05.338353
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d4\d418'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4181.csv
**************************************************
2022-02-24 17:26:51.843420
2022-02-24 17:26:55.401900
2022-02-24 17:27:13.490504
##################################################
4182.csv
**************************************************
2022-02-24 17:27:18.653690
2022-02-24 17:27:22.174270
2022-02-24 17:27:39.432098
##################################################
4183.csv
**************************************************
2022-02-24 17:27:44.573342
2022-02-24 17:27:47.907421
2022-02-24 17:28:04.807207
##################################################
4184.csv
**************************************************
2022-02-24 17:28:09.785886
2022-02-24 17:28:13.158863
2022-02-24 17:28:29.537041
##################################################
4185.csv
**************************************************
2022-02-24 17:28:34.120778
2022-02-24 17:28:37.430922
2022-02-24 17:28:59.059055
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d4\d419'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4191.csv
**************************************************
2022-02-24 17:30:58.619173
2022-02-24 17:31:02.257439
2022-02-24 17:31:19.079433
##################################################
4192.csv
**************************************************
2022-02-24 17:31:24.158841
2022-02-24 17:31:27.336341
2022-02-24 17:31:44.647026
##################################################
4193.csv
**************************************************
2022-02-24 17:31:52.616704
2022-02-24 17:31:55.826116
2022-02-24 17:32:00.743958
##################################################
4194.csv
**************************************************
2022-02-24 17:32:03.510556
2022-02-24 17:32:06.947362
2022-02-24 17:32:12.191333
##################################################
4195.csv
**************************************************
2022-02-24 17:32:14.735524
2022-02-24 17:32:18.629107
2022-02-24 17:32:37.165513
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d4\d420'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
4201.csv
**************************************************
2022-02-24 17:34:29.265591
2022-02-24 17:34:32.360311
2022-02-24 17:34:46.856525
##################################################
4202.csv
**************************************************
2022-02-24 17:34:53.739112
2022-02-24 17:34:58.544262
2022-02-24 17:35:15.934727
##################################################
4203.csv
**************************************************
2022-02-24 17:35:21.867853
2022-02-24 17:35:24.964568
2022-02-24 17:35:40.848071
##################################################
4204.csv
**************************************************
2022-02-24 17:35:47.924649
2022-02-24 17:35:50.902681
2022-02-24 17:36:07.263907
##################################################
4205.csv
**************************************************
2022-02-24 17:36:14.253207
2022-02-24 17:36:17.419735
2022-02-24 17:36:35.079487
########################################